In [1]:
# Importing necessary modules

import sklearn.datasets as datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import tree
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import pandas as pd 
# read text file into pandas DataFrame and 
# create header 
import os
Column_names = ["TIME_str", "TIME_sec", "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
            "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN", "Vertical_label",\
           "Horizontal_label", "Speed_label", "Turn_rate_deg_per_sec", "REL_HEADING", "DIST_TO_DEST", "MANEUVER_ID" ]
Selected_col_names = [ "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN",  \
                      "Turn_rate_deg_per_sec", "REL_HEADING", "DIST_TO_DEST"]
Target_category_name = "Horizontal_label"
data_name = "HOLD_3"
cwd = os.getcwd()
mypath = cwd + f"/../data/{data_name}/"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
count = 0

for file_name in onlyfiles:
    if "Traj" in file_name:
        if count == 0:
            file_path = mypath + file_name
            df = pd.read_csv(file_path, sep=",", header=None)
            df.columns = Column_names
            X_df = df[Selected_col_names]
            Y_target = df[Target_category_name]
        else:
            file_path = mypath + file_name
            df_temp = pd.read_csv(file_path, sep=",", header=None)
            df_temp.columns = Column_names
            
            X_df_temp = df_temp[Selected_col_names]
            Y_target_temp = df_temp[Target_category_name]
            X_df = pd.concat([X_df, X_df_temp])
            Y_target = pd.concat([Y_target, Y_target_temp])
        count +=1
        # if count >= 10:
        #     break
X_df["Turn_rate_deg_per_sec"] = X_df["Turn_rate_deg_per_sec"]*10

In [4]:
target_names =  list(set(list(Y_target)))
word_map_to_num = set(list(Y_target))
print("category:")
word_map_to_num


category:


{' Fly Towards Destination',
 ' Flying opposite established course',
 ' Hold Left Turn -1.5DegPerSec',
 ' Hold Left Turn -3.0DegPerSec',
 ' Hold Right Turn 1.5DegPerSec',
 ' Hold Right Turn 3.0DegPerSec',
 ' Maintain Current Heading',
 ' No Conclusion: Horizontal'}

In [5]:
n_sample = len(X_df)
print("number of points:", n_sample)
X_df = X_df.head(n_sample)
Y_target = Y_target.head(n_sample)
Y = []
for y in Y_target:
    for idx, s in enumerate(word_map_to_num):
        if s == y:
            Y.append(idx)
Y = np.array(Y)

number of points: 54570


In [6]:
import pandas as pd 
import dtreeviz
from sklearn.tree import export_text
# read text file into pandas DataFrame and 
# create header 
import os
cwd = os.getcwd()

# file_name_test = "Traj_AAL80-11121269.dat" # worst
file_name_test = "Traj_AAL245-11147798_L_HOLD.dat" # normal

file_path_test = mypath + file_name_test


# file_name_test = "Traj_AAL1252-11172342POS_HOLD.dat"

# file_path_test = cwd + "/../data/HOLD/" + file_name_test


df_test = pd.read_csv(file_path_test, sep=",", header=None)
df_test.columns = Column_names
X_df_test = df_test[Selected_col_names]

In [7]:
print("number of testing points:", len(X_df_test))

number of testing points: 3407


In [8]:
Z_target = df_test[Target_category_name]
n_sample = len(X_df_test)
X_df_test = X_df_test.head(n_sample)
Z_target = Z_target.head(n_sample)
Z = []
for z in Z_target:
    for idx, s in enumerate(word_map_to_num):
        if s == z:
            Z.append(idx)
Z = np.array(Z)
len(Z)

3407

## target 1: Horizontal_label

# Entropy

In [9]:
# Entropy, small tree with train data

criterion = "entropy"
max_depth = 6
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg

0.7649257833974712


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


In [10]:
# Entropy, small tree with train data
from sklearn.tree import export_text
from sklearn.tree import export_text_in_conditions

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg
r = export_text_in_conditions(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)



0.7649257833974712


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


If GROUNDSPEED_KTS <= 320.50 && REL_HEADING <= -120.14 && Turn_rate_deg_per_sec <= -5.00 && REL_HEADING <= -171.27 && DIST_TO_DEST <= 62.44 && DIST_TO_DEST <= 36.85, then  class:  Flying opposite established course

If GROUNDSPEED_KTS <= 320.50 && REL_HEADING <= -120.14 && Turn_rate_deg_per_sec <= -5.00 && REL_HEADING <= -171.27 && DIST_TO_DEST <= 62.44 && DIST_TO_DEST > 36.85, then  class:  Flying opposite established course

If GROUNDSPEED_KTS <= 320.50 && REL_HEADING <= -120.14 && Turn_rate_deg_per_sec <= -5.00 && REL_HEADING <= -171.27 && DIST_TO_DEST > 62.44, then  class:  Flying opposite established course

If GROUNDSPEED_KTS <= 320.50 && REL_HEADING <= -120.14 && Turn_rate_deg_per_sec <= -5.00 && REL_HEADING > -171.27 && ALTITUDE_FT <= 6299.00 && GROUNDTRACK_DEG <= 255.50, then  class:  Flying opposite established course

If GROUNDSPEED_KTS <= 320.50 && REL_HEADING <= -120.14 && Turn_rate_deg_per_sec <= -5.00 && REL_HEADING > -171.27 && ALTITUDE_FT <= 6299.00 && GROUNDTRACK_DEG 

In [11]:
import graphviz
from sklearn.tree import export_graphviz
dot_data = export_graphviz(dtree_small_entropy,
                           feature_names=X_df.columns,  
                           class_names = target_names,

                           filled=True,  
                           rounded = True,
                            precision = 2,
                           proportion=True)
graph = graphviz.Source(dot_data)
graph.format = "png"
graph.render(f"{data_name}__all_{Target_category_name}_{criterion}" + "importances")

'HOLD_3__all_Horizontal_label_entropyimportances.png'

In [12]:
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)

In [13]:
print(r)

|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- DIST_TO_DEST <= 62.44
|   |   |   |   |   |--- DIST_TO_DEST <= 36.85
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- DIST_TO_DEST >  36.85
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALTITUDE_FT <= 6299.00
|   |   |   |   |   |--- GROUNDTRACK_DEG <= 255.50
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- GROUNDTRACK_DEG >  255.50
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- ALTITUDE_FT >  6299.00
|   |   |   |   |   |--- REL_HEADING <= -154.27
|   |   |   |   |   |   |--- class:  Ho

In [14]:
tree = model.tree_

In [15]:
print(tree.children_left[tree.children_left[2]])

4


In [16]:
criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8725673446948874
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- DIST_TO_DEST <= 62.44
|   |   |   |   |   |--- DIST_TO_DEST <= 36.85
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- DIST_TO_DEST >  36.85
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALTITUDE_FT <= 6299.00
|   |   |   |   |   |--- GROUNDTRACK_DEG <= 255.50
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 197.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  197.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- GROUNDTRACK_DEG >  2

In [17]:
# Entropy, large tree with train data

criterion = "entropy"
dtree_large_n = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_n.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_n, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9981674912955837
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -320.00
|   |   |   |   |   |--- REL_HEADING <= -176.69
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- REL_HEADING >  -176.69
|   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -320.00
|   |   |   |   |   |--- DIST_TO_DEST <= 34.00
|   |   |   |   |   |   |--- REL_HEADING <= -172.69
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- REL_HEADING >  -172.69
|   |   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |   |--- DIST_TO_DEST >  34.00
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALTITUDE_

In [18]:
# Entropy, large tree with test data

criterion = "entropy"
dtree_large_entropy = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9069562665101262
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -320.00
|   |   |   |   |   |--- REL_HEADING <= -176.69
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- REL_HEADING >  -176.69
|   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -320.00
|   |   |   |   |   |--- DIST_TO_DEST <= 34.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 227.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  227.50
|   |   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |   |--- DIST_TO_DEST >  34.00
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALT

In [19]:
# Entropy, small tree with test data

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8121514528911065
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- DIST_TO_DEST <= 62.44
|   |   |   |   |   |--- DIST_TO_DEST <= 36.85
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- DIST_TO_DEST >  36.85
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALTITUDE_FT <= 6299.00
|   |   |   |   |   |--- GROUNDTRACK_DEG <= 255.50
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 197.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  197.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- GROUNDTRACK_DEG >  2

In [20]:
# Entropy, large rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8902260052832404


In [21]:
# Entropy, small rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees, max_depth=5, min_samples_leaf=20)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.634282359847373


# Gini

In [22]:
# Gini, small tree with train data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.7433571559464908
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- DIST_TO_DEST <= 62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALTITUDE_FT <= 6299.00
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- ALTITUDE_FT >  6299.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |--- Turn_rate_deg_per_sec >  -5.00
|   |   |   |--- REL_HEADING <= -124.17
|   |   |   |   |--- DIST_TO_DEST <= 70.87
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  70.87
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |--- REL_HEADING >  -124.17
|   |   |   | 

In [23]:
# Gini, large tree with train data
criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9981674912955837
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -320.00
|   |   |   |   |   |--- REL_HEADING <= -176.69
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- REL_HEADING >  -176.69
|   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -320.00
|   |   |   |   |   |--- DIST_TO_DEST <= 34.00
|   |   |   |   |   |   |--- Turn_rate_deg_per_sec <= -15.00
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- Turn_rate_deg_per_sec >  -15.00
|   |   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |   |--- DIST_TO_DEST >  34.00
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   

In [24]:
# Gini, large tree with test data

criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8990314059289697
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -320.00
|   |   |   |   |   |--- REL_HEADING <= -176.69
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- REL_HEADING >  -176.69
|   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -320.00
|   |   |   |   |   |--- DIST_TO_DEST <= 34.00
|   |   |   |   |   |   |--- REL_HEADING <= -172.69
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- REL_HEADING >  -172.69
|   |   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |   |--- DIST_TO_DEST >  34.00
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALTITUDE_

In [25]:
# Gini, small tree with test data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.5597299677135309
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- DIST_TO_DEST <= 62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALTITUDE_FT <= 6299.00
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- ALTITUDE_FT >  6299.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |--- Turn_rate_deg_per_sec >  -5.00
|   |   |   |--- REL_HEADING <= -124.17
|   |   |   |   |--- DIST_TO_DEST <= 70.87
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  70.87
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |--- REL_HEADING >  -124.17
|   |   |   | 

## Log_loss

In [26]:
# Log_loss, small tree with train data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_Log_loss, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.7417995235477369
|--- GROUNDSPEED_KTS <= 320.50
|   |--- REL_HEADING <= -120.14
|   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |--- REL_HEADING <= -171.27
|   |   |   |   |--- DIST_TO_DEST <= 62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  62.44
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -171.27
|   |   |   |   |--- ALTITUDE_FT <= 6299.00
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- ALTITUDE_FT >  6299.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |--- Turn_rate_deg_per_sec >  -5.00
|   |   |   |--- REL_HEADING <= -124.17
|   |   |   |   |--- REL_HEADING <= -141.81
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- REL_HEADING >  -141.81
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -124.17
|   | 

In [27]:
# Log_loss, large tree with train data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9981674912955837


In [28]:
# Log_loss, large tree with test data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9090108599941297


In [29]:
# Log_loss, small tree with test data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.5826240093924273
